### location file format : map-making-app -> GeoGuessr

In [4]:
import json

In [7]:
# 入力ファイルと出力ファイルのパス
input_file = './input/map-making-app_1.json'
output_file = './output/geoguessr_1.json'

In [8]:
# JSONファイルを読み込んで、customCoordinatesを抽出し、出力する
try:
    with open(input_file, 'r', encoding='utf-8') as infile:
        data = json.load(infile)
    
    # customCoordinatesを抽出
    custom_coordinates = data.get('customCoordinates', None)
    
    if custom_coordinates is None:
        print("customCoordinatesが見つかりませんでした。")
    else:
        # 結果を新しいJSONファイルに書き込む
        with open(output_file, 'w', encoding='utf-8') as outfile:
            json.dump(custom_coordinates, outfile, ensure_ascii=False, indent=4)
        print(f"customCoordinatesが {output_file} に出力されました。")
except FileNotFoundError:
    print(f"{input_file} が見つかりませんでした。")
except json.JSONDecodeError:
    print("JSONファイルの形式が正しくありません。")


customCoordinatesが ./output/geoguessr_1.json に出力されました。
